# 1. 建立索引 (Build Index)
本 Notebook 為主要入口，負責初始化資料庫 Schema、準備文件並建立階層式向量索引。

## 流程總覽
1. 載入 `.env` 並確認連線設定。
2. 初始化階層式 Schema（可重複執行）。
3. 執行階層式 Chunking 與索引。

> 提示：Notebook 只聚焦流程協調，實際邏輯皆封裝在 `rag_system` core library 或 `scripts/` 目錄。

In [ ]:
# Step 1: 載入環境變數與設定
import os
from dotenv import load_dotenv

load_dotenv()
conn = os.environ.get("PGVECTOR_URL")
print(f'Using PGVECTOR_URL={conn}')
assert conn, '請先在 .env 中設定 PGVECTOR_URL'

# 可依需要覆寫模型或 API 端點
embed_api_key = os.environ.get('EMBED_API_KEY')
llm_api_key = os.environ.get('LLM_API_KEY')
print('Embedding key set:', bool(embed_api_key))
print('LLM key set:', bool(llm_api_key))

## Step 2: 初始化階層式 Schema
可在 Notebook 直接呼叫腳本，或在 Terminal 執行：

```bash
python scripts/init_hierarchical_schema.py --conn $PGVECTOR_URL
```

下方範例使用 IPython 的 `!` 呼叫腳本，並可改用 `--verify` 查看結構。

In [ ]:
# 在 Notebook 內初始化資料庫 Schema
!python scripts/init_hierarchical_schema.py --conn $PGVECTOR_URL --verify

## Step 3: 建立階層式索引
使用 `scripts/index_hierarchical.py` 以 Clean Architecture 的 `IndexDocumentUseCase` 處理 chunking 與向量化。

```bash
# 索引單一檔案
python scripts/index_hierarchical.py data/example.md --conn $PGVECTOR_URL

# 或遞迴索引整個資料夾
python scripts/index_hierarchical.py data/ --recursive --conn $PGVECTOR_URL
```

在 Notebook 中可直接執行下列範例，並依需求替換路徑與參數。

In [ ]:
# 將 markdown/txt 檔案轉為階層式索引
!python scripts/index_hierarchical.py data/ --recursive --conn $PGVECTOR_URL --force

### 後續
- 索引完成後，可在 `notebooks/2_query_verify.ipynb` 中載入 Agent 進行檢索與回答驗證。
- 若需要一次性維運或遷移，可使用 `scripts/migrate_to_hierarchical.py` 等工具。